# Parte 1

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import t # t - student 
import os 
import pyreadr

In [ ]:
user = os.getlogin()   # Username
os.chdir(f"C:/Users/user/Documentos/Git_Hub/1ECO35_2022_2/Lab4")

In [ ]:
data = pyreadr.read_r(r"../data/cps2012.Rdata")
type(data)

In [ ]:
data2012 = data['data'] #me permite leer el dataframe

In [ ]:
data2012

In [ ]:
# selecting columns
X = data2012.loc[:,['exp1','exp2','exp3','exp4']] # DataFrame de variables explicativas
y = data2012.iloc[:,1] # El vector de la variable Y

In [ ]:
class RegClass( object ):
    
    __slots__ = [ 'X', 'y', 'intercept','X_np','y_np', 'beta_OLS', 'reg_estandar', 'R2yMSE', 'reg_robusta' , 'Table'] #privatizamos el método que muestra los resultados en un objeto diccionario:
    def __init__( self, X : pd.DataFrame , y : pd.Series,robust_sd = True ): #booleano para errores estándar robustos
    
        if not isinstance( X, pd.DataFrame ):
            raise TypeError( "X must be a pd.DataFrame." )

        if not isinstance( y , pd.Series ):
            raise TypeError( "y must be a pd.Series." )
            
        # asignando atributos de la clase
          
        self.y = y
        self._X = X #Privatizamos el atributo X (dataframe de variables explicativas)
        self.robust_sd = robust_sd
        intercept = True
        self.intercept = intercept
        # agregamos la columna de unos
        if self.intercept:  

        self._X[ 'Intercept' ] = 1 
        cols = self._X.columns.tolist() 
        new_cols_orders = [cols[ -1 ]] + cols[ 0:-1 ] 
        self._X = self._X.loc[ : , new_cols_orders ] 

        else:
            pass
        
        self._X_np = self._X.values  
        self.y_np = y.values.reshape( -1 , 1 ) 
        self.columns = self._X.columns.tolist() 
    
    def beta_OLS( self ):
        
        X_np = self._X_np
        y_np = self.y_np

        # beta_ols
        beta_ols = np.linalg.inv( X_np.T @ X_np ) @ ( X_np.T @ X_np )
        
         # columnas de X
        index_names = self.columns
        
         # Output
        beta_OLS_output = pd.DataFrame( beta_ols , index = index_names , columns = [ 'OLS'] )
        
        # agregamos Dataframe de coeficientes como atributo
        self.beta_OLS = beta_OLS_output
        
        
        return beta_OLS_output
    
    def R2yMSE( self ):
        
        
        y_est =  self._X @ self.beta_ols # y estimado
        self.y_est = y_est 
        error = self.y_np - y_est # vector de errores
        self.SCR = np.sum(np.square(error)) #suma de cuadrados de la regresión
        SCT = np.sum(np.square(self.y_np - np.mean(self.y_np))) #suma total de cuadrados
        N = X.shape[ 0 ] # número de filas
    
        # R2 y RMSE
        R2 = 1 - self.SCR/SCT 
        rmse = (self.SCR/N)**0.5
    
    def reg_estandar( self ):
    
        #llamamos al método anterior
        self.beta_OLS()
        self.R2yMSE()
        
        X_np = self._X_np
        y_np = self.y_np
        
        # beta_ols
        beta_OLS = self.beta_OLS.values.reshape( - 1, 1 ) 

        # errors
        self.e = y_np - ( X_np @ beta_OLS )

        # error variance
        N = X.shape[ 0 ]
        total_parameters = X.shape[ 1 ]
        error_var = ( (e.T @ e)[ 0 ] )/( N - total_parameters )

        # Varianza stándar
        var_OLS =  error_var * np.linalg.inv( X_np.T @ X_np )
        var_OLS_output = pd.DataFrame( var_OLS , index = index_names , columns = index_names )
        self.var_OLS = var_OLS_output
        
         # standard errors
        beta_se = np.sqrt( np.diag( var_OLS ) )
        self.beta.se = beta_se

        # Confidence interval
        
        up_bd = beta_OLS.ravel() + 1.96*beta_se
        lw_bd = beta_OLS.ravel() - 1.96*beta_se

        table_data ={  'Coef.'    : beta_OLS.ravel() ,  # .ravel() :: .flatten()
                       "Std.Err." : beta_se.ravel(),
                       "t"        : t_stat.ravel(),
                       "P>|t|"    : pvalue.ravel(), 
                       "[0.025"   : lw_bd.ravel(),
                       "0.975]"   : up_bd.ravel()
                    }
        return reg_estandar
        
    def reg_robusta( self ):
        
        self.beta_OLS()
        self.reg_estandar()
        self.R2yMSE()
       
        
         # var y beta
        beta_OLS = self.beta_OLS.values.reshape( -1, 1 )
        var_OLS_robust = var_OLS_robust.values
        
         # Varianza robust
        matrix_robust =  np.diag( (self.e.T @ self.e)[ 0 ] )
        var_OLS_robust = np.linalg.inv(self.X2.T @ self.X2) @ self.X2.T @ matrix_robust @ self.X2 @ np.linalg.inv(self.X2.T @ self.X2)
        var_OLS_output_robust = pd.DataFrame( var_OLS_robust , index = index_names , columns = index_names )
        self.var_OLS_robust = var_OLS_output_robust
      
        
        # standard errors
        beta_se_robust = np.sqrt( np.diag( var_OLS_robust ) )

        # Confidence interval
        
        up_bd = beta_OLS.ravel() + 1.96*beta_se_robust
        lw_bd = beta_OLS.ravel() - 1.96*beta_se_robust

       
        return reg_robusta
    
    def Table(self, **Kargs):
        
        # Se corren las funciones anteriores
        self.beta_OLS()
        self.reg_estandar()
        self.R2yMSE()     
        

        if self.robust:
            
            reg_OLS = pd.DataFrame( {"OLS": self.beta_OLS.flatten() , "standar_error" : self.beta_se} )
                    
            
        
            diccionario ={"OLS": self.beta_OLS.flatten() , "standar_error" : self.beta_se ,
                                        "variance" : self.var_OLS.flatten() , "lower_bound" : lw_bd.flatten() , "upper_bound" : up_bd.flatten() }
        
        else:

                
             reg_OLS = pd.DataFrame( {"OLS": self.beta_OLS.flatten() , "standar_error" : beta_se} )
            
             diccionario ={"OLS": self.beta_OLS.flatten() , "standar_error" : self.beta_se,
                                        "variance" : self.var_OLS.flatten() , "lower_bound" : lw_bd.flatten() , "upper_bound" : up_bd.flatten() }
        return diccionario

# Parte 2


In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame,Series
import os
import pyreadstat

In [2]:
user = os.getlogin()
os.chdir(f"C:/Users/user/Documentos/Git_Hub/1ECO35_2022_2/Lab6")
data_administrativa = pd.read_spss( r"../data/data_administrativa.sav" )#subimos la base de datos

In [3]:
data_administrativa 

,year,MES,CONGLOME,VIVIENDA,HOGAR,CODPERSO,UBIGEO,DOMINIO,ESTRATO,P201P,P203,P203A,P203B,P204,P205,P206,P207,P208A,P208B,P209
0,2020,06,010108,136,11,02,100111,Sierra Centro,De 100 000 a 499 999 habitantes,20200101081361102,Esposo(a)/compañero(a),NaN,NaN,Si,No,NaN,Mujer,43.0,NaN,Casado(a)
1,2020,06,020257,093,11,04,250107,Selva,De 100 000 a 499 999 habitantes,20200202570931104,Hijo(a)/Hijastro(a),NaN,NaN,Si,No,NaN,Hombre,17.0,NaN,Soltero(a)
2,2020,06,015537,065,11,04,040126,Sierra Sur,De 500 000 a más habitantes,20200155370651104,Hijo(a)/Hijastro(a),NaN,NaN,Si,No,NaN,Mujer,9.0,NaN,NaN
3,2019,10,010614,043,11,07,130901,Sierra Norte,De 20 000 a 49 999 habitantes,20190106140431107,Otros parientes,2.0,Hijo(a),No,NaN,No,Mujer,4.0,NaN,NaN
4,2020,06,009390,096,11,04,230110,Costa Sur,De 100 000 a 499 999 habitantes,20180093900961104,Panel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85030,2020,12,007337,004,11,01,140105,Costa Norte,De 100 000 a 499 999 habitantes,20200073370041101,Jefe/Jefa,1.0,Jefe/jefa de hogar,Si,No,NaN,Hombre,65.0,NaN,Casado(a)
85031,2019,04,008226,084,13,02,160112,Selva,De 100 000 a 499 999 habitantes,20190082260841302,Otros no parientes,0.0,Jefe/jefa de hogar,Si,No,NaN,Mujer,89.0,NaN,Viudo(a)
85032,2019,05,008965,060,11,03,210101,Sierra Sur,De 100 000 a 499 999 habitantes,20190089650601103,Hijo(a)/Hijastro(a),1.0,Hijo(a),Si,No,NaN,Hombre,20.0,NaN,Soltero(a)
85033,2020,05,010311,022,11,02,200501,Costa Norte,De 50 000 a 99 999 habitantes,20200103110221102,Esposo(a)/compañero(a),NaN,NaN,Si,No,NaN,Mujer,48.0,NaN,Casado(a)


In [4]:
import savReaderWriter as sav  # import sav (package)


with sav.SavHeaderReader( r"../data/data_administrativa.sav", ioUtf8=True) as header:
    metadata = header.all() # save dataset
    labels_data_administrativa = metadata.valueLabels  # get labels from varaibles varaibles's values 
    var_labels_data_administrativa = metadata.varLabels # get labels from varaibles 
    
 # ioUtf8 read special characters    

In [5]:
# Variables que presentan valores faltantes
data_administrativa.isna().any()

year        False
MES         False
CONGLOME    False
VIVIENDA    False
HOGAR       False
CODPERSO    False
UBIGEO      False
DOMINIO     False
ESTRATO     False
P201P       False
P203        False
P203A        True
P203B        True
P204         True
P205         True
P206         True
P207         True
P208A        True
P208B        True
P209         True
dtype: bool

In [6]:
#las etiquetas de las variables de la base de datos

var_labels_data_administrativa 

{'year': '',
 'MES': '',
 'CONGLOME': '',
 'VIVIENDA': '',
 'HOGAR': '',
 'CODPERSO': '',
 'UBIGEO': '',
 'DOMINIO': 'Dominio geográfico',
 'ESTRATO': 'Estrato geográfico',
 'P201P': '',
 'P203': '¿Cuál es la relación de parentesco con el jefe(a) del hogar',
 'P203A': '',
 'P203B': 'Relación de parentesco con el jefe(a) del núcleo familiar',
 'P204': '¿Es miembro del hogar?',
 'P205': '¿Se encuentra ausente del hogar 30 días o más?',
 'P206': '¿Está presente en el hogar 30 días o más?',
 'P207': 'Sexo',
 'P208A': '',
 'P208B': '',
 'P209': '¿Cuál es su estado civil o conyugal?'}

In [7]:
var_labels_data_administrativa['DOMINIO'] #la etiqueta de la variable 'DOMINIO' de la base de datos

'Dominio geográfico'

In [8]:
var_labels_data_administrativa['P203'] #la etiqueta de la variable 'P203' de la base de datos

'¿Cuál es la relación de parentesco con el jefe(a) del hogar'

In [9]:
type(labels_data_administrativa) # value's labels 

dict

In [10]:
#las etiquetas de los valores de las variables en la base de datos

labels_data_administrativa 

{'DOMINIO': {1.0: 'Costa Norte',
  2.0: 'Costa Centro',
  3.0: 'Costa Sur',
  4.0: 'Sierra Norte',
  5.0: 'Sierra Centro',
  6.0: 'Sierra Sur',
  7.0: 'Selva',
  8.0: 'Lima Metropolitana'},
 'ESTRATO': {1.0: ' De 500 000 a más habitantes',
  2.0: ' De 100 000 a 499 999 habitantes',
  3.0: ' De 50 000 a 99 999 habitantes',
  4.0: ' De 20 000 a 49 999 habitantes',
  5.0: 'De 2 000 a 19 999 habitantes',
  6.0: ' De 500 a 1 999 habitantes',
  7.0: ' Área de Empadronamiento Rural (AER) Compuesto',
  8.0: ' Área de Empadronamiento Rural (AER) Simple'},
 'P203': {0.0: 'Panel',
  1.0: 'Jefe/Jefa',
  2.0: 'Esposo(a)/compañero(a)',
  3.0: 'Hijo(a)/Hijastro(a)',
  4.0: 'Yerno/Nuera',
  5.0: 'Nieto(a)',
  6.0: 'Padres/Suegros',
  7.0: 'Otros parientes',
  8.0: 'Trabajador Hogar',
  9.0: 'Pensionista',
  10.0: 'Otros no parientes',
  11.0: 'Hermano(a)'},
 'P203B': {1.0: 'Jefe/jefa de hogar',
  2.0: 'Esposa(o)',
  3.0: 'Hijo(a)',
  6.0: 'Padres/suegros'},
 'P204': {1.0: 'Si', 2.0: 'No'},
 'P205': {1

In [11]:
labels_data_administrativa['DOMINIO'] #las etiquetas de los valores de la variable 'DOMINIO'

{1.0: 'Costa Norte',
 2.0: 'Costa Centro',
 3.0: 'Costa Sur',
 4.0: 'Sierra Norte',
 5.0: 'Sierra Centro',
 6.0: 'Sierra Sur',
 7.0: 'Selva',
 8.0: 'Lima Metropolitana'}

In [12]:
labels_data_administrativa['P203'] #las etiquetas de los valores de la variable 'P203'

{0.0: 'Panel',
 1.0: 'Jefe/Jefa',
 2.0: 'Esposo(a)/compañero(a)',
 3.0: 'Hijo(a)/Hijastro(a)',
 4.0: 'Yerno/Nuera',
 5.0: 'Nieto(a)',
 6.0: 'Padres/Suegros',
 7.0: 'Otros parientes',
 8.0: 'Trabajador Hogar',
 9.0: 'Pensionista',
 10.0: 'Otros no parientes',
 11.0: 'Hermano(a)'}

In [13]:
# Adding labels to variables enapres2020_1 from pandas 

data_administrativa.attrs[ 'var_labels' ] = var_labels_data_administrativa # var labels
data_administrativa.attrs[ 'value_labels' ] = labels_data_administrativa # value's labels 

In [14]:
# Duplicados a partir del identificador por persona : conglome, vivienda, hogar y codperso

data_administrativa[ data_administrativa.loc[:, ['CODPERSO', 'CONGLOME' , 'VIVIENDA', 'HOGAR']].duplicated() ] 
#variables que tienen duplicados

,year,MES,CONGLOME,VIVIENDA,HOGAR,CODPERSO,UBIGEO,DOMINIO,ESTRATO,P201P,P203,P203A,P203B,P204,P205,P206,P207,P208A,P208B,P209
2632,2020,01,006571,174,11,02,100402,Sierra Centro,Área de Empadronamiento Rural (AER) Compuesto,20190065711741103,Hijo(a)/Hijastro(a),1.0,Hijo(a),Si,No,NaN,Hombre,17.0,NaN,Soltero(a)
2724,2019,10,005278,048,11,04,021701,Sierra Centro,De 2 000 a 19 999 habitantes,20190052780481104,Hijo(a)/Hijastro(a),1.0,Hijo(a),Si,No,NaN,Mujer,9.0,NaN,NaN
2773,2020,12,005190,037,11,01,021801,Costa Centro,De 100 000 a 499 999 habitantes,20190051900371101,Jefe/Jefa,1.0,Jefe/jefa de hogar,Si,No,NaN,Mujer,72.0,NaN,Conviviente
3135,2019,12,006886,168,11,02,120114,Sierra Centro,De 100 000 a 499 999 habitantes,20190068861681102,Esposo(a)/compañero(a),1.0,Esposa(o),Si,No,NaN,Mujer,38.0,NaN,Casado(a)
3663,2019,02,008611,048,11,04,190113,Sierra Centro,De 50 000 a 99 999 habitantes,20190086110481104,Hijo(a)/Hijastro(a),1.0,Hijo(a),Si,No,NaN,Mujer,4.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84940,2019,12,008253,047,11,01,160113,Selva,De 100 000 a 499 999 habitantes,20170082530471101,Jefe/Jefa,1.0,Jefe/jefa de hogar,Si,No,NaN,Hombre,80.0,NaN,Conviviente
84945,2019,06,007982,008,11,02,150202,Costa Centro,De 20 000 a 49 999 habitantes,20170079820081102,Hijo(a)/Hijastro(a),1.0,Hijo(a),Si,No,NaN,Hombre,19.0,NaN,Soltero(a)
84975,2020,09,007221,043,11,03,130901,Sierra Norte,De 20 000 a 49 999 habitantes,20170072210431103,Yerno/Nuera,NaN,NaN,Si,No,NaN,Hombre,38.0,NaN,Conviviente
84982,2019,07,008806,114,11,04,200701,Costa Norte,De 50 000 a 99 999 habitantes,20170088061141104,Nieto(a),2.0,Hijo(a),Si,No,NaN,Hombre,6.0,NaN,NaN


In [15]:
data_administrativa[ data_administrativa.loc[:, ['CODPERSO', 'CONGLOME' , 'VIVIENDA', 'HOGAR']].duplicated(keep = False) ] 
#variables que tienen duplicados y sus duplicados

,year,MES,CONGLOME,VIVIENDA,HOGAR,CODPERSO,UBIGEO,DOMINIO,ESTRATO,P201P,P203,P203A,P203B,P204,P205,P206,P207,P208A,P208B,P209
3,2019,10,010614,043,11,07,130901,Sierra Norte,De 20 000 a 49 999 habitantes,20190106140431107,Otros parientes,2.0,Hijo(a),No,NaN,No,Mujer,4.0,NaN,NaN
5,2019,03,008010,059,11,03,150801,Costa Centro,De 100 000 a 499 999 habitantes,20190080100591103,Otros parientes,1.0,Hijo(a),Si,No,NaN,Hombre,32.0,NaN,Soltero(a)
24,2019,07,009625,071,11,01,250105,Selva,De 100 000 a 499 999 habitantes,20190096250711101,Jefe/Jefa,1.0,Jefe/jefa de hogar,Si,No,NaN,Hombre,64.0,NaN,Conviviente
33,2020,07,010368,081,11,04,230104,Costa Sur,De 100 000 a 499 999 habitantes,20180103680811105,Hijo(a)/Hijastro(a),NaN,NaN,Si,No,NaN,Hombre,14.0,NaN,Soltero(a)
52,2019,12,005867,034,11,02,060101,Sierra Norte,De 100 000 a 499 999 habitantes,20170058670341102,Esposo(a)/compañero(a),1.0,Esposa(o),Si,No,NaN,Mujer,53.0,NaN,Casado(a)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84940,2019,12,008253,047,11,01,160113,Selva,De 100 000 a 499 999 habitantes,20170082530471101,Jefe/Jefa,1.0,Jefe/jefa de hogar,Si,No,NaN,Hombre,80.0,NaN,Conviviente
84945,2019,06,007982,008,11,02,150202,Costa Centro,De 20 000 a 49 999 habitantes,20170079820081102,Hijo(a)/Hijastro(a),1.0,Hijo(a),Si,No,NaN,Hombre,19.0,NaN,Soltero(a)
84975,2020,09,007221,043,11,03,130901,Sierra Norte,De 20 000 a 49 999 habitantes,20170072210431103,Yerno/Nuera,NaN,NaN,Si,No,NaN,Hombre,38.0,NaN,Conviviente
84982,2019,07,008806,114,11,04,200701,Costa Norte,De 50 000 a 99 999 habitantes,20170088061141104,Nieto(a),2.0,Hijo(a),Si,No,NaN,Hombre,6.0,NaN,NaN


In [16]:
# Base de datos ordenada a partir de las variables que identifican cada miembro y la variable de año (year)

data_administrativa.sort_values(["CODPERSO","year"], ascending = [True,True]) 
data_administrativa

,year,MES,CONGLOME,VIVIENDA,HOGAR,CODPERSO,UBIGEO,DOMINIO,ESTRATO,P201P,P203,P203A,P203B,P204,P205,P206,P207,P208A,P208B,P209
0,2020,06,010108,136,11,02,100111,Sierra Centro,De 100 000 a 499 999 habitantes,20200101081361102,Esposo(a)/compañero(a),NaN,NaN,Si,No,NaN,Mujer,43.0,NaN,Casado(a)
1,2020,06,020257,093,11,04,250107,Selva,De 100 000 a 499 999 habitantes,20200202570931104,Hijo(a)/Hijastro(a),NaN,NaN,Si,No,NaN,Hombre,17.0,NaN,Soltero(a)
2,2020,06,015537,065,11,04,040126,Sierra Sur,De 500 000 a más habitantes,20200155370651104,Hijo(a)/Hijastro(a),NaN,NaN,Si,No,NaN,Mujer,9.0,NaN,NaN
3,2019,10,010614,043,11,07,130901,Sierra Norte,De 20 000 a 49 999 habitantes,20190106140431107,Otros parientes,2.0,Hijo(a),No,NaN,No,Mujer,4.0,NaN,NaN
4,2020,06,009390,096,11,04,230110,Costa Sur,De 100 000 a 499 999 habitantes,20180093900961104,Panel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85030,2020,12,007337,004,11,01,140105,Costa Norte,De 100 000 a 499 999 habitantes,20200073370041101,Jefe/Jefa,1.0,Jefe/jefa de hogar,Si,No,NaN,Hombre,65.0,NaN,Casado(a)
85031,2019,04,008226,084,13,02,160112,Selva,De 100 000 a 499 999 habitantes,20190082260841302,Otros no parientes,0.0,Jefe/jefa de hogar,Si,No,NaN,Mujer,89.0,NaN,Viudo(a)
85032,2019,05,008965,060,11,03,210101,Sierra Sur,De 100 000 a 499 999 habitantes,20190089650601103,Hijo(a)/Hijastro(a),1.0,Hijo(a),Si,No,NaN,Hombre,20.0,NaN,Soltero(a)
85033,2020,05,010311,022,11,02,200501,Costa Norte,De 50 000 a 99 999 habitantes,20200103110221102,Esposo(a)/compañero(a),NaN,NaN,Si,No,NaN,Mujer,48.0,NaN,Casado(a)


In [ ]:
#Cree una base de datos para cada año

#Creamos la base de datos para el año 2019
data_administrativa_2019_Grupo10 = data_administrativa.loc[ (data_administrativa.year == '2019'), : ]

#Guardamos la base de datos
pyreadstat.write_sav(data_administrativa, r"../../data/data_administrativa_2019_Grupo10.sav")

In [ ]:
#Creamos la base de datos para el año 2020
data_administrativa_2020_Grupo10 = data_administrativa.loc[ (data_administrativa.year == '2020'), : ]

#Guardamos la base de datos
pyreadstat.write_sav(data_administrativa, r"../../data/data_administrativa_2020_Grupo10.sav")